In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2 as cv
from numpy.random import seed
seed(45)
import pickle

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob 
from tensorflow.keras import layers
from tensorflow.keras import backend as K

%matplotlib inline

## Transfer learning with fine-tuning -- week7
version7

## Helper Functions
The functions below are used for merging the contents of Keras history objects, and for displaying training curves. We will use these after each training run.

In [ ]:
def merge_history(hlist):
    history = {}
    for k in hlist[0].history.keys():
        history[k] = sum([h.history[k] for h in hlist], [])
    return history

def vis_training(h, start=1):
    epoch_range = range(start, len(h['loss'])+1)
    s = slice(start-1, None)

    plt.figure(figsize=[14,4])

    n = int(len(h.keys()) / 2)

    for i in range(n):
        k = list(h.keys())[i]
        plt.subplot(1,n,i+1)
        plt.plot(epoch_range, h[k][s], label='Training')
        plt.plot(epoch_range, h['val_' + k][s], label='Validation')
        plt.xlabel('Epoch'); plt.ylabel(k); plt.title(k)
        plt.grid()
        plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
train_dirname = '/kaggle/input/histopathologic-cancer-detection/train'

# Dataset exploration

In [ ]:
train_labels = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')
train_labels.head()

# Label distribution

In [ ]:
train_labels['label'].value_counts()

In [ ]:
# Display a DataFrame showing the proportion of observations with each 
# possible of the target variable (which is label). 
(train_labels.label.value_counts() / len(train_labels)).to_frame()

In [ ]:
train_labels.info()

Data is not entirely balanced, there is more negative samples than positive, by about 30 percent

# View Sample Images

In [ ]:
positive_samples = train_labels.loc[train_labels['label'] == 1].sample(4)
negative_samples = train_labels.loc[train_labels['label'] == 0].sample(4)
positive_images = []
negative_images = []
for sample in positive_samples['id']:
    path = os.path.join(train_dirname, sample+'.tif')
    img = cv.imread(path)
    positive_images.append(img)
        
for sample in negative_samples['id']:
    path = os.path.join(train_dirname, sample+'.tif')
    img = cv.imread(path)
    negative_images.append(img)

fig,axis = plt.subplots(2,4,figsize=(20,8))
fig.suptitle('Dataset samples presentation plot',fontsize=20)
for i,img in enumerate(positive_images):
    axis[0,i].imshow(img)
    rect = patches.Rectangle((32,32),32,32,linewidth=4,edgecolor='g',facecolor='none', linestyle=':', capstyle='round')
    axis[0,i].add_patch(rect)
axis[0,0].set_ylabel('Positive samples', size='large')
for i,img in enumerate(negative_images):
    axis[1,i].imshow(img)
    rect = patches.Rectangle((32,32),32,32,linewidth=4,edgecolor='r',facecolor='none', linestyle=':', capstyle='round')
    axis[1,i].add_patch(rect)
axis[1,0].set_ylabel('Negative samples', size='large')
    

# Splitting dataset

# Setting up learning constants

In [ ]:
IMG_SIZE = 96
IMG_CHANNELS = 3
#TRAIN_SIZE=80000
TRAIN_SIZE = 10000
BATCH_SIZE = 64
EPOCHS = 30

## Balancing the dataset

In [ ]:
train_neg = train_labels[train_labels['label']==0].sample(TRAIN_SIZE,random_state=45)
train_pos = train_labels[train_labels['label']==1].sample(TRAIN_SIZE,random_state=45)

train_data = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

train_data = shuffle(train_data)

In [ ]:
train_data['label'].value_counts()

In [ ]:
def append_ext(fn):
    return fn+".tif"

## Splitting the dataset

In [ ]:
#y = train_data['label']
#train_df, val_df = train_test_split(train_data, test_size=0.3, random_state=45, stratify=y)
#y = val_df['label']
#val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=45, stratify=y)
#print(train_df.shape)
#print(val_df.shape)
#print(test_df.shape)

In [ ]:
y = train_data['label']
train_df, valid_df = train_test_split(train_data, test_size=0.2, random_state=45, stratify=y)

print(train_df.shape)
print(valid_df.shape)

In [ ]:
train_df['id'] = train_df['id'].apply(append_ext)
valid_df['id'] = valid_df['id'].apply(append_ext)
train_df.head()

## Image generators for the simple CNN model

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
BATCH_SIZE = 64
train_path = '../input/histopathologic-cancer-detection/train'
train_df['label'] = train_df['label'].astype(str)
valid_df['label'] = valid_df['label'].astype(str)

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (64,64)
)

valid_loader = valid_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (64,64)
)

In [ ]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

## Build Network

In this section, we will construct our neural network. For feature extraction, we will use the VGG16 model, as trained on the ImageNet dataset.

In the cell below, we will load the pretrained VGG16 model into a variable named base_model. We will set include_top=False to indicate that we only wish to use the convolutional blocks that appear before the Flatten() layer. We will not include the dense layers composing the classifier at the top of the network. Instead, we will design and train our own classifier.

We set the input_shape parameter to indicate the shape of the images that we will be feeding into the network.

Finally, we set the trainable parameter of the model to False. This tells Keras that we do not wish to update the weights in the base layer during training. We only wish to train the new classifier that we will design.

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(64,64,3),
                                         include_top=False,
                                         weights='imagenet')

base_model.trainable = False

Before moving forward, let's take a look at the structure of our base model. Notice that it consists of 5 convolutional blocks, some of which contain 2 convolutional layers, and some of which contain 3. Also note that none of the weights in the model are trainable (since we have set them to not be).

In [ ]:
base_model.summary()

VGG16 is one of many pretrained models that we could have used. Common choices include VGG16, VGG19, ResNet50, and InceptionV3. A full list of the pretrained models provided by Keras can be found here: Keras Applications

We are now ready to build a classifier for our neural network. In the cell below, we include base_model in the network as if were a single layer.

In [ ]:
cnn = Sequential([
    base_model,
    
    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.25),
    BatchNormalization(),
    Dense(2, activation='softmax')
])

cnn.summary()

## Train Network

In [ ]:
opt = tf.keras.optimizers.Adam(0.001)
cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

## Training Run 1

In [ ]:
%%time 

h1 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 15,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
history = h1.history
print(history.keys())

In [ ]:
epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#Training Run 2
tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)

In [ ]:
%%time 

h2 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 15,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
for k in history.keys():
    history[k] += h2.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

## Fine-Tuning the Model

### Fine-tuning is a technique that allows us to adjust the layers in a pretrained convolutional base to let it adapt to the new dataset it is being applied to. This is accomplished by unfreezing the layers in the base, setting a low learning rate, and then training for some number of additional epochs.

In [ ]:
base_model.trainable = True
K.set_value(cnn.optimizer.learning_rate, 0.00001)
cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

### Notice that when we view a summary of the model, we see that the number of trainable parameters has increased significantly. This is the result of unfreezing the convolutional base.

In [ ]:
cnn.summary()

### We will now train all layers of the model with a low learning rate.

In [ ]:
h3 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    epochs = 10,
    verbose = 1
)

In [ ]:
print(history.keys())
print(h3.history.keys())

In [ ]:
print(h3.history)

h3.history['auc'] = h3.history['auc_1']
del h3.history['auc_1']

h3.history['val_auc'] = h3.history['val_auc_1']
del h3.history['val_auc_1']
print(h3.history)

In [ ]:
for k in history.keys():
    history[k] += h3.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

### Save Model and History

In [ ]:
cnn.save('cancer_detection_model_v16.h5')
pickle.dump(history, open(f'cancer_detection_history_v16.pkl', 'wb'))

## 